In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import torchvision
import scipy as scp
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sys
sys.path.insert(1, '/home/kseuro/Kai/deeplearnphysics/pytorch/particle_generator/')

# Meitner Machine
import ae
import conv_ae
import ewm
import utils
from dataloader import LArCV_loader

#### Set the GPU to be used for model evaluation
- On Meitner, GPU 1 is the best option

In [ ]:
device = torch.device(1)

#### Set the root path of the AutoEncoder Experiments Folder

In [ ]:
exp_root = "/media/hdd1/kai/particle_generator/experiments/"

#### Select the model class and append to the experiment root

In [ ]:
mlp = False
if mlp:
    AE_root = exp_root + 'larcv_ae/mlp_ae/'
    EWM_root = exp_root + 'ewm_models/mlp_ewm/'
else:
    AE_root = exp_root + 'larcv_ae/conv_ae/'
    EWM_root = exp_root + 'ewm_models/conv_ewm/'

In [ ]:
# Path to model weights
weights_dir = "weights/"

# Select the AutoEncoder and Generator Models you want to deploy

## Get the names of all the saved AutoEncoder experiments in the exp_root folder

In [ ]:
AE_paths = []
for path in os.listdir(AE_root):
    AE_paths.append(os.path.join(AE_root, path))

print("-"*60)
for i in range(len(AE_paths)):
    AE_name = AE_paths[i].split('/')[-1]
    print("\n Exp_{}:".format(str(i)), AE_name, '\n')
    print("-"*60)

## Select the AutoEncoder experiment you want

In [ ]:
AE_dir = AE_paths[1]

In [ ]:
# Create the full path to the AutoEncoder experiment
AE_path = os.path.join(AE_root, AE_dir) + "/"
print("Path to AutoEncoder set as: \n{}".format(AE_path))

## Get the names of all the saved EWM Generator models

In [ ]:
EWM_paths = []
# EWM_root += 'evaluated_models/'
for path in os.listdir(EWM_root):
    EWM_paths.append(os.path.join(EWM_root, path))

print("-"*60)
for i in range(len(EWM_paths)):
    EWM_name = EWM_paths[i].split('/')[-1]
    print("\n Exp_{}:".format(str(i)), EWM_name, '\n')
    print("-"*60)

## Select the EWM model experiment you want

In [ ]:
EWM_dir = EWM_paths[2]

In [ ]:
# Create the full path to the EWM model
EWM_path = os.path.join(EWM_root, EWM_dir) + "/"
print("Path to EWM Generator Model set as: \n{}".format(EWM_path))

# Load selected models from checkpoint and onto GPU

## Load the AE config .csv as a dict
- Get the model architecture from the config_df

In [ ]:
config_csv = AE_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

In [ ]:
# Get the model architecture from config df
num_epochs = int(config_df[config_df['Unnamed: 0'].str.contains("epochs")==True]['0'].values.item())
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].values.item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].values.item())
depth    = int(config_df[config_df['Unnamed: 0'].str.contains("depth")==True]['0'].values.item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].values.item())**2
im_dim   = int(np.sqrt(im_size))

## Set up AutoEncoder model on the GPU

### Compute the layer dimensions for the AutoEncoder
- TODO: Need to add function that deals with an AutoEncoder Model trained on MNIST

In [ ]:
# Set up AE layer sizes
if 'mlp' in AE_root:    
    base = [256] 

    # Compute encoder sizes
    sizes = lambda: [ (yield 2**i) for i in range(n_layers) ]
    enc_sizes = base * n_layers
    enc_sizes = [a*b for a,b in zip(enc_sizes, [*sizes()])][::-1]

    # Update kwarg dicts
    # Decoder is the reverse of the encoder
    ae_kwargs = {'enc_sizes' : enc_sizes, 'l_dim' : l_dim, 'im_size' : im_size, 'dec_sizes' : enc_sizes[::-1]}
else:
    # Compute the depth of the feature maps, based on the number of
    # specified layers. If depth is not divisibe by 4, warn
    depth   = [depth] * n_layers
    divisor = lambda: [ (yield 2**i) for i in range(n_layers) ]
    depth   = [a//b for a,b in zip(depth, [*divisor()])][::-1]
        
    # Update kwarg dicts
    # Decoder is the reverse of the encoder
    ae_kwargs = {'enc_depth':[1] + depth, 'dec_depth':depth[1:len(depth)][::-1] + [1],'l_dim':l_dim}

In [ ]:
print("Model l_dim: {}".format(l_dim))
print("Encoder depth: {}".format(ae_kwargs['enc_depth']))
print("Decoder depth: {}".format(ae_kwargs['dec_depth']))

In [ ]:
# Set up model on GPU
if 'mlp' in AE_root:
    model = ae.AutoEncoder(**ae_kwargs).to(device)
else:
    model = conv_ae.ConvAutoEncoder(**ae_kwargs).to(device)

## Load the AE model checkpoint

In [ ]:
# Get checkpoint name(s)
AE_checkpoint_path  = AE_path + weights_dir
AE_checkpoint_names = []
for file in os.listdir(AE_checkpoint_path):
    AE_checkpoint_names.append(os.path.join(AE_checkpoint_path, file))

In [ ]:
print("-"*60)
for i in range(len(AE_checkpoint_names)):
    name = AE_checkpoint_names[i].split('/')[-1]
    AE_checkpoint_names.append(name)
    print("\n{}:".format(str(i)), name, '\n')
    print("-"*60)

In [ ]:
# Select the desired checkpoint from the list
AE_checkpoint = AE_checkpoint_names[2]

In [ ]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(AE_checkpoint)

In [ ]:
# Load the model's state dictionary
# Note: The IncompatibleKeys(missing_keys=[], unexpected_keys=[]) message indicates that
#       there were no problems in loading the state dictionary. Bit confusing...
model.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Put the model in evaluation mode
model.eval()

### If evaluating convolutional model, get the size of the code tensor using a random torch tensor

In [ ]:
if 'conv' in AE_root:
    x = torch.randn(1, 1, im_dim, im_dim).to(device)
    x = model.encoder(x)
    code = (x.shape[1], x.shape[2], x.shape[3])
    print("Code tensor volume is: [{} x {} x {}] = {}".format(code[0], code[1], code[2], code[0]*code[1]*code[2]))

In [ ]:
x.shape

## Load the G config .csv as a dict
- Get the model architecture from the config_df

In [ ]:
config_csv = EWM_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

In [ ]:
# Get the model architecture from config df
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].values.item())
n_hidden = int(config_df[config_df['Unnamed: 0'].str.contains("n_hidden")==True]['0'].values.item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].values.item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].values.item())
z_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("z_dim")==True]['0'].values.item())
print("{} Layer model with {} hidden units per layer".format(n_layers, n_hidden))
if 'mlp' in EWM_root:
    print("Mapping {}_dim_Gaussian to {}_dim code vectors".format(z_dim, l_dim))
else:
    print("Mapping {}_dim_Gaussian to {}_dim code vectors".format(z_dim, code[0]*code[1]*code[2]))

## Set up the Generator model

In [ ]:
# Model kwargs
fc_sizes = [n_hidden] * n_layers
ewm_kwargs = { 'z_dim': z_dim, 'fc_sizes': fc_sizes, 'n_out': code[0]*code[1]*code[2]}
# fc_sizes = [512] * 4
# ewm_kwargs = { 'z_dim': 100, 'fc_sizes': fc_sizes, 'n_out': 384}

In [ ]:
# Create generator on GPU
G = ewm.ewm_G(**ewm_kwargs).to(device)

## Load EWM model checkpoint

In [ ]:
# Get checkpoint name(s)
EWM_checkpoint_path  = EWM_path + weights_dir
EWM_checkpoint_names = []
for file in os.listdir(EWM_checkpoint_path):
    EWM_checkpoint_names.append(os.path.join(EWM_checkpoint_path, file))

In [ ]:
print("-"*60)
for i in range(len(EWM_checkpoint_names)):
    name = EWM_checkpoint_names[i].split('/')[-1]
    print("\n {} :".format(str(i)), name, '\n')
    print("-"*60)

In [ ]:
# Select the checkpoint you want
EWM_checkpoint = EWM_checkpoint_names[0]

In [ ]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
# checkpoint = torch.load(EWM_checkpoint)
checkpoint = torch.load(EWM_checkpoint)

In [ ]:
# Load the model's state dictionary
# Note: The IncompatibleKeys(missing_keys=[], unexpected_keys=[]) message indicates that
#       there were no problems in loading the state dictionary. Bit confusing...
G.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Put the model in evaluation mode
G.eval()

## Check that the l_dim of the AE is the same as the out_dim of the Generator

In [ ]:
if 'mlp' in AE_root:
    ae_l_dim = AE.encoder.last.out_features
else:
    ae_l_dim = code[0]*code[1]*code[2]
g_out_dim = G.out[0].out_features

In [ ]:
if ae_l_dim != g_out_dim:
    print("Code vector sizes do not match!")
    print("AE: {} | G: {}".format(ae_l_dim, g_out_dim))
else:
    print("Code vector sizes match")

#### Compute the arguments for reshaping the Generator's output

In [ ]:
model.decoder.deconv_blocks[0][2].in_channels
model.decoder.deconv_blocks[0][2].out_channels

In [ ]:
in_f  = model.decoder.deconv_blocks[0][2].in_channels
out_f = model.decoder.deconv_blocks[0][2].out_channels
h = code[1]
w = code[2]
print('[{},{},{},{}]'.format(in_f, out_f, h, w))

## Deploy the Decoder using G as an input

In [ ]:
z_dim = 100
mean = []; var = []
for i in range(1000):
    out = G(torch.randn(1, 1, z_dim).to(device))
    out = out.detach().cpu().numpy()
    mean.append(np.mean(out))
    var.append(np.var(out))

In [ ]:
print(np.mean(mean), np.mean(var))

In [ ]:
%matplotlib inline
plt.plot(mean, label='mean')
plt.plot(var, label='var')
plt.legend()

In [ ]:
samples1 = []; samples2 = []; limit = 16

In [ ]:
for itr in range(limit):
    # Create an input vector for G
    z_rand = torch.randn(2, 1, z_dim).to(device)
    
    # Get a code vector from G
    code_vec = G(z_rand).view(2, in_f, h, w).to(device)
    
    # Push code_vec through Decoder
    sample = model.decoder(code_vec)
    
    # Reshape the sample, detach, and convert to numpy array
    samples1.append(sample[0].view(im_dim, im_dim).detach().cpu().numpy())
    samples2.append(sample[1].view(im_dim, im_dim).detach().cpu().numpy())

In [ ]:
%matplotlib inline
plt.imshow(samples1[0], cmap='gray')

In [ ]:
plt.imshow(samples1[6], cmap='gray')

## Viz the deploy samples

In [ ]:
%matplotlib inline
# Setup figure
fig, axes = plt.subplots(nrows = 2, ncols = 8, sharex=True, sharey=True, figsize=(25,6))

# Set title
if 'mlp' in AE_root:
    m_label = 'AE'
else:
    m_label = 'Conv_AE'

# Generate plot
for images, row in zip([samples1, samples2], axes):
    for img, ax in zip(images, row):
        img = np.reshape(img, (im_dim, im_dim))
        ax.imshow(img, cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax.tick_params(axis='x', colors='#443941')
        ax.tick_params(axis='y', colors='#443941')
fig.tight_layout()

# Save figure
save_file = "ewm_ae_{}_{}_{}.png".format(im_dim, code[0]*code[1]*code[2], m_label)
plt.savefig(save_file, dpi=300)